In [1]:
import pytorch_lightning as pl
from  data.data_loader import MyDataModule 
from models import OneLayer
from losses.CrossEntropy import CrossEntropy
from losses.Tversky import FocalTverskyLoss
from losses.Dice import DiceLoss
from losses.Focal import FocalLoss
from losses.Jacard import JacardLoss
from training.PL_train import Main_Loop
from metrics.accuracy import accuracy


In [2]:
model = OneLayer(n_classes=5).cuda()
criterion = JacardLoss().cuda()
data_module = MyDataModule(prepaire=False)

In [3]:
trainer = pl.Trainer(gpus=1,max_epochs = 100)
main = Main_Loop(model = model,loss= criterion, metric= accuracy)
trainer.fit(main, data_module)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
  2%|▏         | 30/1251 [00:07<05:32,  3.68it/s]